In [1]:
# Install python client resources

!pip3 install --pre -I "weaviate-client==4.*"


  Using cached requests-2.31.0-py3-none-any.whl.metadata (4.6 kB)
  Using cached validators-0.22.0-py3-none-any.whl.metadata (4.7 kB)
  Using cached Authlib-1.2.1-py2.py3-none-any.whl.metadata (3.8 kB)
  Using cached pydantic-2.5.0-py3-none-any.whl.metadata (174 kB)
  Using cached grpcio-1.59.2-cp310-cp310-macosx_12_0_universal2.whl.metadata (4.0 kB)
  Using cached grpcio_tools-1.59.2-cp310-cp310-macosx_12_0_universal2.whl.metadata (6.2 kB)
  Using cached cryptography-41.0.5-cp37-abi3-macosx_10_12_universal2.whl.metadata (5.2 kB)
  Using cached protobuf-4.25.0-cp37-abi3-macosx_10_9_universal2.whl.metadata (541 bytes)
  Using cached setuptools-68.2.2-py3-none-any.whl.metadata (6.3 kB)
  Using cached annotated_types-0.6.0-py3-none-any.whl.metadata (12 kB)
  Using cached pydantic_core-2.14.1-cp310-cp310-macosx_11_0_arm64.whl.metadata (6.5 kB)
  Using cached typing_extensions-4.8.0-py3-none-any.whl.metadata (3.0 kB)
  Using cached charset_normalizer-3.3.2-cp310-cp310-macosx_11_0_arm64.whl.

True

In [1]:
# OLD CLIENT
# OLD CLIENT
# OLD CLIENT
# OLD CLIENT

import weaviate
from weaviate import EmbeddedOptions
import os

client = weaviate.Client(
    url = "http://localhost:8080/",  # Replace with your endpoint
    additional_headers = {
        "X-OpenAI-Api-Key": os.getenv("OPENAI_API_KEY")# Replace with your inference API key
    }
)

client.is_ready()

True

In [3]:
# OLD CLIENT

class_definition = {
    
    "class": "Question",
    "vectorizer":"text2vec-openai",
    "vectorIndexConfig": {
        "distance" : "cosine"
    },
    
    'properties' : [
        {
            'name' : "question",
            "dataType" : ['text']
        },
        {
            'name' : "answer",
            "dataType" : ["text"]
        },
        {
            'name' : 'round',
            'dataType': ['text']
        }
    ]
}

client.schema.create_class(class_definition)

UnexpectedStatusCodeException: Create class! Unexpected status code: 422, with response body: {'error': [{'message': 'class name "Question" already exists'}]}.

In [6]:
# BOTH

import requests
import json

# Download the data
resp = requests.get('https://raw.githubusercontent.com/weaviate-tutorials/intro-workshop/main/data/jeopardy_1k.json')

# Load the data so you can see what it is
data = json.loads(resp.text)

# Parse the JSON and preview it
print(type(data), len(data))
print(json.dumps(data[1], indent=2))

<class 'list'> 1000
{
  "Air Date": "2005-11-18",
  "Round": "Jeopardy!",
  "Value": 200,
  "Category": "RHYME TIME",
  "Question": "Any pigment on the wall so faded you can barely see it",
  "Answer": "faint paint"
}


In [7]:
# OLD CLIENT


with client.batch() as batch:
    for o in data:
        obj_body = {
            'question':o["Question"],
            'answer':o["Answer"],
            'round':o["Round"]
        }
        
        batch.add_data_object(
        data_object=obj_body,
        class_name="Question"
        )


/Users/davecuthbert/src/weaviate-io/.venv/lib/python3.10/site-packages/weaviate/warnings.py:131: DeprecationWarning: Dep006: You are using the `client.batch()` method, which will be removed in the next major release.
            Please instead use the `client.batch.configure()` method to configure your batch and `client.batch` to enter the context manager.
            See https://weaviate.io/developers/weaviate/client-libraries/python for details.
  warnings.warn(


In [8]:
# OLD CLIENT

# Test upload

print(json.dumps(client.query.aggregate("Question").with_meta_count().do(), indent=2))



{
  "data": {
    "Aggregate": {
      "Question": [
        {
          "meta": {
            "count": 2000
          }
        }
      ]
    }
  }
}


In [9]:
# OLD CLIENT

client.schema.update_config(
   "Question", {
      "vectorIndexConfig": {
         # highlight-start
         "pq": {
            "enabled": True,         # Enable PQ 
            "trainingLimit": 100000, 
            "segments": 96 
         # highlight-end
         }
      }
   }
)

In [10]:
# OLD CLIENT

client.schema.get()

{'classes': [{'class': 'Question',
   'invertedIndexConfig': {'bm25': {'b': 0.75, 'k1': 1.2},
    'cleanupIntervalSeconds': 60,
    'stopwords': {'additions': None, 'preset': 'en', 'removals': None}},
   'moduleConfig': {'text2vec-openai': {'baseURL': 'https://api.openai.com',
     'model': 'ada',
     'modelVersion': '002',
     'type': 'text',
     'vectorizeClassName': True}},
   'multiTenancyConfig': {'enabled': False},
   'properties': [{'dataType': ['text'],
     'indexFilterable': True,
     'indexSearchable': True,
     'moduleConfig': {'text2vec-openai': {'skip': False,
       'vectorizePropertyName': False}},
     'name': 'question',
     'tokenization': 'word'},
    {'dataType': ['text'],
     'indexFilterable': True,
     'indexSearchable': True,
     'moduleConfig': {'text2vec-openai': {'skip': False,
       'vectorizePropertyName': False}},
     'name': 'answer',
     'tokenization': 'word'},
    {'dataType': ['text'],
     'indexFilterable': True,
     'indexSearchable':

In [1]:
# NEW PY4 CLIENT
# NEW PY4 CLIENT
# NEW PY4 CLIENT
# NEW PY4 CLIENT


# Connect to instance 

import weaviate, os, json
import weaviate.classes as wvc

client = weaviate.connect_to_local(
    headers={
        "X-OpenAI-Api-Key": os.environ['OPENAI_API_KEY']  # Replace with your inference API key
    }
)

client.is_ready()

True

In [3]:
 client.collections.delete("Question")

In [4]:
import requests
import json

# Download the data
resp = requests.get('https://raw.githubusercontent.com/weaviate-tutorials/intro-workshop/main/data/jeopardy_1k.json')

# Load the data so you can see what it is
data = json.loads(resp.text)

# Parse the JSON and preview it
print(type(data), len(data))
print(json.dumps(data[1], indent=2))

<class 'list'> 1000
{
  "Air Date": "2005-11-18",
  "Round": "Jeopardy!",
  "Value": 200,
  "Category": "RHYME TIME",
  "Question": "Any pigment on the wall so faded you can barely see it",
  "Answer": "faint paint"
}


In [5]:
import weaviate, os, json
import weaviate.classes as wvc

client = weaviate.connect_to_local(
    headers={
        "X-OpenAI-Api-Key": os.environ['OPENAI_API_KEY']  # Replace with your inference API key
    }
)

client.is_ready()

True

In [6]:
# Define collection

client.collections.create(
    name="JeopardyCategory",
    description="A Jeopardy! category",
    vectorizer_config=wvc.Configure.Vectorizer.text2vec_openai(),
    generative_config=wvc.Configure.Generative.openai(),
    properties=[
        wvc.Property(name="title", data_type=wvc.DataType.TEXT),
    ]
)

In [12]:
dataList = [ obj for obj in data]
print(dataList[0])

{'Air Date': '2006-11-08', 'Round': 'Double Jeopardy!', 'Value': 800, 'Category': 'AMERICAN HISTORY', 'Question': 'Abraham Lincoln died across the street from this theatre on April 15, 1865', 'Answer': "Ford's Theatre (the Ford Theatre accepted)"}


In [27]:
# Batch load data

# with client.batch() as batch:
#     for o in data:
#         obj_body = {
#             'question':o["Question"],
#             'answer':o["Answer"],
#             'round':o["Round"]
#         }
        
#         batch.add_data_object(
#         data_object=obj_body,
#         class_name="Question"
#         )

# client.batch.configure

def parseData():
  objectList = []
  for obj in data:
     objectList.append( { "question": obj["Question"], "answer": obj["Answer"], "round": obj["Round"] } )
     
  return(objectList)
  
jeopardy = client.collections.get("JeopardyCategory")
upload = jeopardy.data.insert_many( parseData() )

#print(upload)


In [28]:
resp = jeopardy.query.bm25(query = 'food')

print(resp)                    

_QueryReturn(objects=[_Object(properties={'answer': 'food stores (supermarkets)', 'question': 'This type of retail store sells more shampoo & makeup than any other', 'round': 'Double Jeopardy!'}, metadata=_MetadataReturn(uuid=UUID('7f47c461-91f3-4f66-aab7-d6a1b28d03b1'), vector=None, creation_time_unix=1700010643827, last_update_time_unix=1700010643827, distance=None, certainty=None, score=2.761819362640381, explain_score=', BM25F_food_frequency:1, BM25F_food_propLength:3', is_consistent=False)), _Object(properties={'answer': 'pasta', 'question': 'Type of food that comes in shapes of bow ties, elbows & wagon wheels', 'round': 'Jeopardy!'}, metadata=_MetadataReturn(uuid=UUID('e48313fd-d979-442c-9413-540e6cebf6e1'), vector=None, creation_time_unix=1700010648216, last_update_time_unix=1700010648216, distance=None, certainty=None, score=1.3178938627243042, explain_score=', BM25F_food_frequency:1, BM25F_food_propLength:12', is_consistent=False)), _Object(properties={'answer': 'Black Forest 

In [29]:
response = jeopardy.aggregate.over_all(total_count=True)

print(response.total_count)

1000


In [30]:
def parseData():
  objectList = []
  for obj in data:
     objectList.append( 
        {
            "question": obj["Question"], 
            "answer": obj["Answer"], 
            "round": obj["Round"] 
         }
     )
     
  return(objectList)
  
jeopardy = client.collections.get("JeopardyCategory")
jeopardy.data.insert_many( parseData() )

# Check upload
response = jeopardy.aggregate.over_all(total_count=True)

# Should equal number of objects uploaded
print(response.total_count)


2000


In [66]:
# Update schema
import weaviate.classes as wvc

jeopardy = client.collections.get("JeopardyCategory")
jeopardy.config.update(
   vector_index_config=wvc.Reconfigure.vector_index(
      pq_enabled=True,
      pq_segments=96,
      pq_training_limit=100000
   )
)



In [32]:
response = jeopardy.aggregate.over_all(total_count=True)

# Should equal number of objects uploaded
print(response.total_count)

2000


In [79]:
# Get schema definitions

jeopardy = client.collections.get( "JeopardyCategory" )
config = jeopardy.config.get()
pq_config = config.vector_index_config.pq

# print some of the config properties
print( f"Enabled: { pq_config.enabled }" )
print( f"Training: { pq_config.training_limit }" )
print( f"Segments: { pq_config.segments }" )
print( f"Centroids: { pq_config.centroids }" )


Enabled: True
Training: 100000
Segments: 96
Centroids: 256


In [77]:
client = weaviate.connect_to_local(
    port=8080,
    grpc_port=50051,
    headers={
        "X-OpenAI-Api-Key": os.environ[ "OPENAI_API_KEY" ] # Replace with your inference API key
    }
)

In [70]:
### OLD CLIENT

import weaviate
from weaviate import EmbeddedOptions
import os

old_client = weaviate.Client(
    url = "http://localhost:8080/",  # Replace with your endpoint
    additional_headers = {
        "X-OpenAI-Api-Key": os.getenv("OPENAI_API_KEY")# Replace with your inference API key
    }
)

old_client.is_ready()

True

In [74]:
old_client.schema.get()

{'classes': [{'class': 'JeopardyCategory',
   'description': 'A Jeopardy! category',
   'invertedIndexConfig': {'bm25': {'b': 0.75, 'k1': 1.2},
    'cleanupIntervalSeconds': 60,
    'stopwords': {'additions': None, 'preset': 'en', 'removals': None}},
   'moduleConfig': {'generative-openai': {},
    'text2vec-openai': {'baseURL': 'https://api.openai.com',
     'model': 'ada',
     'modelVersion': '002',
     'type': 'text',
     'vectorizeClassName': True}},
   'multiTenancyConfig': {'enabled': False},
   'properties': [{'dataType': ['text'],
     'indexFilterable': True,
     'indexSearchable': True,
     'moduleConfig': {'text2vec-openai': {'skip': False,
       'vectorizePropertyName': True}},
     'name': 'title',
     'tokenization': 'word'},
    {'dataType': ['number'],
     'description': "This property was generated by Weaviate's auto-schema feature on Wed Nov 15 00:41:26 2023",
     'indexFilterable': True,
     'indexSearchable': False,
     'moduleConfig': {'text2vec-openai':